# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
import sys

In [77]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser

In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
cmap='jet'

In [ ]:
base_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
dic_dir = {"190130" : ["absC"],
           "190131" : ["absAHP12", "absB", "absD", "absJ"],
           "190201" : ["absA", "absAHP20", "absE", "absG", "absM"],
           "190204" : ["absAHP9", "absAHP30", "absAHP45", "absH", "absI", "absK", "absL", "absN", "absO", "absU"]}

In [6]:
dir_A     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190201/absA"
dir_C     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190128/absC"
dir_AHP9  = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP9"
dir_AHP45 = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP45"

# Fond de chambre

In [7]:
dir_fondDeChambre = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
tmp = np.loadtxt("S11_fond_de_chambre.txt", skiprows=1)
S11_fondDeChambre = tmp[:,1] + 1j * tmp[:,2]

In [8]:
fig, ax = plt.subplots(1,1)
plt.plot( 20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre ) ) ), label="fond de chambre" )
plt.grid()
plt.legend()

# Plaque de référence

In [12]:
dir_plaqueDeReference = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190129/caracterisation/plaque_ref"
freq_all, S11_plaque_ref = ser.getFreqAndS11( dir_plaqueDeReference )

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

## Initialisation des variables

In [13]:
nb_elev = 41
nb_freq = 2001

elevation        = np.linspace( 0.76 - 1.0, 0.76 + 1.0, nb_elev )
data_S11         = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_td      = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_tg      = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_td = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_tg = np.zeros( (nb_elev, nb_freq), dtype=complex )
reflectivite     = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_moy          = np.zeros( nb_freq,            dtype=complex )
S11_moy_square   = np.zeros( nb_freq,            dtype=complex )

## Lecture des données et time gating

In [91]:
seq = 0
flag_gate = 1
data_dir = dir_AHP45

for ielev in range(nb_elev):
    freq_all, S11_all = ser.getFreqAndS11( data_dir, seq0=seq, verbose=0 )
    seq = seq + 3
    data_S11[ ielev, : ] = S11_all
    
d = np.linspace( 0, 300 / ( freq_all[1] - freq_all[0] ), len(freq_all) ) * 0.5

In [92]:
gate = ser.getWindow( d, center, width, id="llc" )

In [97]:
S11_moy     = np.average( data_S11, axis=0 )
S11_moy_td  = np.fft.ifft( S11_moy )
S11_moy_tg  = S11_moy_td * gate
data_S11_td = np.fft.ifft( data_S11, axis=1 )
data_S11_tg = data_S11_td * gate
data_S11_tg_kaiser = data_S11_td * window

In [98]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( data_S11_td[20,:] ) ), label="S11_moy_td" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg[20,:] ) ), label="S11_moy_tg" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg_kaiser[20,:] ) ), label="S11_moy_tg_kaiser" )
ax.legend()
ax.grid()

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


## Gate design

scipy.signal.kaiser(M, beta, sym=True)

**SYM** When True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis

In [505]:
myGate = np.zeros( d.shape )
myWidth = 500
gateSize = np.where( np.abs( d - center ) < myWidth )[0].shape[0]
#myGate[ 0 : gateSize ] = 1.0
myGate[ np.where( np.abs( d - center ) < myWidth ) ] = 1.0
fftMyGate = np.fft.fft( myGate )
print( f"gateSize {gateSize}" )

gateSize 166


In [506]:
fig, (ax0, ax1) = plt.subplots(2,1)

ax0.plot( d, myGate )
ax0.grid()

ax1.plot( 20 * np.log10( np.abs( fftMyGate ) ) )
ax1.grid()

In [590]:
N = 2001
beta = 6
myWindow = np.zeros(nb_freq)
myWindow[0] = 1
myWindow[1:] = np.fft.fftshift( kaiser(N-1, beta, sym=True) )
modMyWindow = 20 * np.log10( np.abs( myWindow ) )

In [594]:
idx = np.where( modMyWindow > -6 )
show = np.ones( nb_freq ) - 35
show[ idx ] = -6
normalization = np.ones( nb_freq )
normalization[idx] = 1 / myWindow[idx]
modNormalization = 20 * np.log10( np.abs( normalization ) )
normalized = myWindow * normalization
modNormalized = 20 * np.log10( np.abs( normalized ) )

In [595]:
fig, ax = plt.subplots(1,1)
ax.plot( fftMyWindow, label = 'fftMyWindow' )
ax.plot( modNormalization, label = 'normalization' )
ax.plot( modNormalized, label = 'normalized' )
ax.plot( np.arange(nb_freq), show, 'r' )
plt.grid()
plt.legend()

In [603]:
fftMyGateKaiser = np.fft.fft( myGate ) * myWindow
myGateKaiser = np.fft.ifft( fftMyGateKaiser )

In [600]:
fig, (ax0, ax1) = plt.subplots(2,1)
ax0.plot( myWindow, '.', label='myWindow' )
ax0.grid()
ax0.legend()

ax1.plot( 20 * np.log10( np.abs( fftMyGate ) ), label='fftshift fftMyGate' )
ax1.plot( 20 * np.log10( np.abs( fftMyGateKaiser ) ), label='fftMyGateKaiser' )
ax1.grid()
ax1.legend()

In [604]:
fig, ax = plt.subplots(1,1)
ax.plot( 20 * np.log10( np.abs( np.real( myGateKaiser ) ) ), '-', label='real' )
ax.plot( 20 * np.log10( np.abs( np.imag( myGateKaiser ) ) ), '-', label='imag' )
ax.plot( 20 * np.log10( np.abs(          myGateKaiser ) ), '.', label='abs')
plt.grid()
plt.legend()

In [563]:
NKaiser = 200
littleKaiser = kaiser(2*NKaiser, 6, sym=True)
myWindowLittleKaiser = np.zeros(nb_freq)
myWindowLittleKaiser[0] = 1
myWindowLittleKaiser[1:NKaiser+1]      = littleKaiser[NKaiser:]
myWindowLittleKaiser[nb_freq-NKaiser:] = littleKaiser[0:NKaiser]
fftMyGateLittleKaiser = fftMyGate  * myWindowLittleKaiser
myGateLittleKaiser = np.fft.ifft( fftMyGateLittleKaiser )

In [564]:
fig, ax = plt.subplots(1,1)
ax.plot( 20 * np.log10( np.abs( np.real( myGateLittleKaiser ) ) ), '-', label='real' )
ax.plot( 20 * np.log10( np.abs( np.imag( myGateLittleKaiser ) ) ), '-', label='imag' )
ax.plot( 20 * np.log10( np.abs(          myGateLittleKaiser ) ), '.', label='abs')
plt.grid()
plt.legend()

In [558]:
fig, ax = plt.subplots(1,1)
ax.plot( 20 * np.log10( np.abs( myGateKaiser ) ), '.-', label='myGateKaiser')
ax.plot( 20 * np.log10( np.abs( myGateLittleKaiser ) ), '.-', label='myGateLittleKaiser')
plt.grid()
plt.legend()

In [525]:
unity = np.ones( nb_freq )
unityGated = unity * myGateLittleKaiser
fftUnityGated = np.fft.fft( unityGated )

In [526]:
fig, (ax0, ax1) = plt.subplots(2,1)

ax0.plot( d, 20 * np.log10( np.abs( unityGated ) ), label='unityGated' )
ax0.grid()
ax0.legend()

ax1.plot( 20 * np.log10( np.abs( fftUnityGated ) ), label = 'fftUnityGated' )
ax1.grid()
ax1.legend()

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


## Post normalization function

In [486]:
unityFResponseKaiser = kaiser( nb_freq, 12, sym=False )

In [487]:
# convolution with the gate response
ifftUnityFResponseKaiser = np.fft.ifft( unityFResponseKaiser )
conv = np.fft.fft( ifftUnityFResponseKaiser * myGateKaiser )

In [489]:
fig, ax = plt.subplots(1,1)
ax.plot( 20 * np.log10( np.abs( ifftUnityFResponseKaiser ) ), '.-', label='conv')
plt.grid()
plt.legend()

In [477]:
fig, ax = plt.subplots(1,1)
ax.plot( 20 * np.log10( np.abs( conv ) ), '.-', label='conv')
ax.plot( 20 * np.log10( np.abs( 1/conv ) ), '.-', label='1/conv')
plt.grid()
plt.legend()

# Filter design

beta Window shape

* 0 	 Rectangular

* 5 	 Similar to a Hamming

* 6 	 Similar to a Hanning

* 8.6  Similar to a Blackman

$$\beta=\alpha\pi$$

In [59]:
center = 6650
width = 500
beta = 3 * np.pi
window = np.zeros( nb_freq )
idx = np.where( np.abs( d - center ) < width )

In [60]:
window[idx] = np.kaiser(idx[0].shape[0], beta)

In [62]:
fig, ax = plt.subplots(1,1)
ax.plot(window)
plt.grid()

In [39]:
fig, (ax0, ax1) = plt.subplots(2,1)

ax0.plot(gate_b0, label='gate_b0')
ax0.plot(gate_b, label='gate_b')
ax0.plot(gate_c, label='gate_c')
ax0.legend()
ax0.grid()

ax1.plot(gate_a0, label='gate_a0')
#ax1.plot(gate_a, label='gate_a')
ax1.legend()
ax1.grid()

In [32]:
offset

30125.69359020951

## Kaiser filter

### a

In [99]:
fL = 0.1  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
fH = 0.4  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
b  = 0.08  # Transition band, as a fraction of the sampling rate (in (0, 0.5)).
N  = int(np.ceil((4 / b)))
if not N % 2:
    N += 1  # Make sure that N is odd.
n  = np.arange(N)
 
# Compute a low-pass filter with cutoff frequency fH.
hlpf = np.sinc(2 * fH * (n - (N - 1) / 2))
hlpf = hlpf * np.blackman(N)
hlpf = hlpf / np.sum(hlpf)
 
# Compute a high-pass filter with cutoff frequency fL.
hhpf = np.sinc(2 * fL * (n - (N - 1) / 2))
hhpf = hhpf * np.blackman(N)
hhpf = hhpf / np.sum(hhpf)
hhpf = -hhpf
hhpf[(N - 1) // 2] += 1
 
# Convolve both filters.
h = np.convolve(hlpf, hhpf)

In [104]:
fig, ax = plt.subplots(1,1)
ax.plot(np.abs(np.fft.fft(h)))

### b

In [107]:
fs = 1000.0
cutoff = 175
width = 24

In [108]:
numtaps, beta = kaiserord(65, width/(0.5*fs))
numtaps, beta

(167, 6.20426)

In [109]:
taps = firwin(numtaps, cutoff, window=('kaiser', beta), scale=False, nyq=0.5*fs)

In [110]:
w, h = freqz(taps, worN=8000)
w *= 0.5*fs/np.pi  # Convert w to Hz.

In [111]:
ideal = w < cutoff  # The "ideal" frequency response.
deviation = np.abs(np.abs(h) - ideal)
deviation[(w > cutoff - 0.5*width) & (w < cutoff + 0.5*width)] = np.nan

In [112]:
plt.figure()
plt.plot(w, 20*np.log10(np.abs(deviation)))
plt.xlim(0, 0.5*fs)
plt.ylim(-90, -60)
plt.grid(alpha=0.25)
plt.axhline(-65, color='r', ls='--', alpha=0.3)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Deviation from ideal (dB)')
plt.title('Lowpass Filter Frequency Response')
plt.show()

In [130]:
plt.figure()
plt.plot(w, 20*np.log10(np.abs(h)),'o')
plt.xlim(0, 0.5*fs)
#plt.ylim(-90, -60)
plt.grid(alpha=0.25)
plt.axhline(-65, color='r', ls='--', alpha=0.3)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Deviation from ideal (dB)')
plt.title('Lowpass Filter Frequency Response')
plt.show()

### c

In [491]:
print(f"center {center}, width {myWidth}, gate size {gateSize}")

center 6650, width 500, gate size 166


In [501]:
k = np.arange( nb_freq )

In [502]:
tc = 166

In [503]:
H = 2 * tc * np.sinc( 2 * tc * k )

In [504]:
fig, ax = plt.subplots( 1, 1 )
ax.plot( 20 * np.log10( np.abs( H ) ) )

## Calcul de la réflectivité

In [85]:
S11_plaque_ref_tg = np.fft.fft( np.fft.ifft( S11_plaque_ref - S11_fondDeChambre ) * gate )
tab_freq = freq_all
reflectivite = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = data_S11 - S11_fondDeChambre
S11_minus_moy_td = np.fft.ifft( S11_minus_moy, axis=1 )
S11_minus_moy_tg = np.fft.fft( S11_minus_moy_td * gate, axis=1 )
reflectivite     = S11_minus_moy_tg / S11_plaque_ref_tg

## Relecture des données de la réflectivité

In [86]:
tmp = np.loadtxt(f"{dir_AHP45}/Reflectivite_absAHP45.txt", skiprows=1)
refAbsAHP45 = tmp[:,1:]
refAbsAHP45_f = tmp[:,0]

In [17]:
vmin = -120
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), refAbsAHP45, vmin=vmin, vmax=vmax, cmap=cmap )

title = f"refAbsAHP45"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [18]:
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), 20*np.log10(np.abs(reflectivite)).T, 
                    vmin=vmin, vmax=vmax, cmap=cmap )

title = f"réflectivite"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [87]:
fig, ax = plt.subplots( 1, 1 )
ax.plot( refAbsAHP45[:,0], label="refAbsAHP45" )
ax.plot( 20*np.log10(np.abs(reflectivite)).T[:,0], label="réflectivite" )
ax.legend()
ax.grid()

# Plot

## Coefficient de réflexion

In [20]:
cmap='jet'

toPlot = 20 * np.log10( np.abs( ( S11_minus_moy_tg ) ) )

valMin = -140
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( freq_all, elevation, toPlot, cmap='jet' )
dia.addColorBar(  im, ax, aspect='auto' )
title = f"Coefficient de reflexion absorbant\n{data_dir}"
ax.set_title(title)
ax.set_ylabel( "Elévation [°]" )
ax.set_xlabel( "Fréquence [GHz]" )

valMax = -78.81 valMin = -140.00


Text(0.5, 0, 'Fréquence [GHz]')

## Réflectivité de l'absorbant

In [22]:
toPlot = np.zeros( (len(elevation), len(tab_freq)) )
gatebis  = 0.0 * gate
gatebis[ np.where( np.abs( freq_all - 62.5 ) < 12.4 ) ] = 1.0
for ielev in range( len( elevation ) ):
    toPlot[ ielev, : ] = 20 * np.log10( 1e-12 + np.abs( reflectivite[ ielev, : ] * gatebis ) )

valMin = -90
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")
     
vmin = -80
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), np.rot90(toPlot), cmap=cmap )

title = f"Reflectivité absorbant\n{data_dir}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

valMax = -59.88 valMin = -90.00


# Gating

In [252]:
mu, sigma = 0, 0.1 # mean and standard deviation
nbPoints = 2000
nbUnit = 100
if 0:
    s = np.zeros(nbPoints) + np.random.normal(mu, sigma, nbPoints)
else:
    s = np.zeros(nbPoints)
s[500] = 1
s[-500] = 1
ifft_s = np.fft.ifft(s)

In [253]:
fig, ax = plt.subplots(1,1)
#ax.plot(20*np.log10(np.abs(np.fft.ifftshift(ifft_s))))
ax.plot(np.real(ifft_s))
#ax.set_ylim(-100,0)

In [254]:
ifft_s

array([ 0.001-0.j,  0.   -0.j, -0.001-0.j, ...,  0.   +0.j, -0.001+0.j,
        0.   +0.j])

In [164]:
1/np.amax(np.abs(ifft_s))

200.01000000000002

In [222]:
minTime = -10
maxTime = 10
Fs = 10
Ns = ( maxTime - minTime ) * Fs + 1
time = np.linspace( minTime, maxTime, Ns )
myGate = np.zeros(time.shape)
myGate[ np.where( np.abs(time) < 1 ) ] = 1
myGateFFT         = np.fft.fftshift( np.fft.fft( myGate ) )
myGateFFTKaiser_a = myGateFFT * np.kaiser( Ns, 6 )
myGateFFTKaiser_b = myGateFFT * np.kaiser( Ns, 3 )
myNewGate_a = np.fft.ifft( np.fft.fftshift( myGateFFTKaiser_a ) )
myNewGate_b = np.fft.ifft( np.fft.fftshift( myGateFFTKaiser_b ) )

In [223]:
fig, ax = plt.subplots(1,1)
#ax.plot(time, 20 * np.log10( np.abs( myGate ) ), label='myGate')
ax.plot(time, 20 * np.log10( np.abs( myNewGate_a ) ), label='myNewGate_a')
ax.plot(time, 20 * np.log10( np.abs( myNewGate_b ) ), label='myNewGate_b')
plt.legend()
plt.grid()

In [199]:
fig, ax = plt.subplots(1,1)
ax.plot(20*np.log10(np.abs(myGateFFT)), label='myGateFFT')
ax.plot(20*np.log10(np.abs(myGateFFTKaiser)), label='myGateFFTKaiser')
plt.legend()
plt.grid()

## Kaiser

numtaps, beta = scipy.signal.kaiserord(ripple, width) Determine the filter window parameters for the Kaiser window method.

beta = scipy.signal.kaiser_beta(a) Compute the Kaiser parameter beta, given the attenuation a.

a = scipy.signal.kaiser_atten(numtaps, width) Compute the attenuation of a Kaiser FIR filter.

**RIPPLE** Upper bound for the deviation (in dB) of the magnitude of the filter’s frequency response from that of the desired filter (not including frequencies in any transition intervals). That is, if w is the frequency expressed as a fraction of the Nyquist frequency, A(w) is the actual frequency response of the filter and D(w) is the desired frequency response, the design requirement is that: abs(A(w) - D(w))) < 10**(-ripple/20)

**WIDTH** of transition region, normalized so that 1 corresponds to pi radians / sample. That is, the frequency is expressed as a fraction of the Nyquist frequency.

In [356]:
from scipy.signal import kaiser_atten, kaiser_beta, kaiserord, kaiser
from scipy.signal import firwin, freqz

In [269]:
kaiser_atten(211, 0.0375)

64.48099630593983

In [274]:
print( "beta values depending on desired sidelobe levels" )
print( f"min {kaiser_beta(48):.2f}, norm {kaiser_beta(68):.2f}, wide {kaiser_beta(57):.2f}, max {kaiser_beta(70):.2f}")

beta values depending on desired sidelobe levels
min 4.31, norm 6.53, wide 5.32, max 6.76


According to https://community.keysight.com/thread/26609 the order of the window is fixed to 6 for gating.

In [281]:
ripple = 10**(0.1/20)

In [282]:
ripple_dB = 20*np.log10(ripple)

In [264]:
passBandRipple = 0.005
rejection_dB = 65
passBandRipple_dB = 20 * np.log10( passBandRipple )
print( f"pass band ripple = {passBandRipple_dB:.2f} [dB], rejection = {rejection_dB} [dB]" )

pass band ripple = -46.02 [dB], rejection = 65 [dB]


In [255]:
fs = 1000.0
cutoff = 175
fs = 
ripple = 65
width = 24
numtaps, beta = kaiserord( ripple, width/(0.5*fs))
numtaps, beta

(167, 6.20426)

In [256]:
taps = firwin(numtaps, cutoff, window=('kaiser', beta), scale=False, nyq=0.5*fs)

In [257]:
w, h = freqz(taps, worN=8000)
w *= 0.5 * fs / np.pi  # Convert w to Hz.

In [258]:
fig, ax = plt.subplots(1,1)
ax.plot( w, 20 * np.log10( np.abs( h ) ), label='h'  )
plt.grid()
plt.legend()

## Windowing
The window settings balance response resolution versus dynamic range.

* Minimum Window = Best Response Resolution
* Maximum Window = Best Dynamic Range


* Kaiser Beta  Changes window size using a Kaiser Beta value
* Impulse Width  Changes window size using an Impulse Width value

Minimum ±0.1 dB  -48 dB  1.4/Freq Span  2.8/Freq Span

Normal  ±0.1 dB  -68 dB  2.8/Freq Span  5.6/Freq Span

Wide    ±0.1 dB  -57 dB  4.4/Freq Span  8.8/Freq Span

Maximum ±0.01 dB -70 dB 12.7/Freq Span 25.4/Freq Span